In [7]:
from character import ALL_PROGRESSIONS, TWO_WEAPONS_CROSSBOWS_NO_FEATS, TWO_WEAPONS_CROSSBOWS_SS_VANILLA
import simulator

raw_data = simulator.simulate_combat(
    ALL_PROGRESSIONS,
    # [TWO_WEAPONS_CROSSBOWS_NO_FEATS, TWO_WEAPONS_CROSSBOWS_SS_VANILLA],
    target_ac=18,
    iterations=5000
)

In [8]:
import pandas as pd

df = pd.DataFrame(raw_data)
df

,iteration_number,round_number,name,level,round_damage
0,0,1,Ranger_Archery_SS_Vanilla,1,0
1,0,2,Ranger_Archery_SS_Vanilla,1,0
2,0,3,Ranger_Archery_SS_Vanilla,1,14
3,0,4,Ranger_Archery_SS_Vanilla,1,0
4,0,5,Ranger_Archery_SS_Vanilla,1,5
...,...,...,...,...,...
1199995,4999,1,Ranger_TwoCrossbows_SS_Vanilla,12,39
1199996,4999,2,Ranger_TwoCrossbows_SS_Vanilla,12,40
1199997,4999,3,Ranger_TwoCrossbows_SS_Vanilla,12,38
1199998,4999,4,Ranger_TwoCrossbows_SS_Vanilla,12,38


In [9]:
df_per_round_stats = df.groupby(['iteration_number', 'level', 'name']).agg(
    mean_round_damage=('round_damage', 'mean'),
    # median_round_damage=('round_damage', 'median'),
    # max_round_damage=('round_damage', 'max'),
    # percentile_10_round_damage=('round_damage', lambda x: x.quantile(0.1)),
    # percentile_90_round_damage=('round_damage', lambda x: x.quantile(0.9))
).reset_index()

df_per_round_stats


,iteration_number,level,name,mean_round_damage
0,0,1,Ranger_Archery_NoFeats,6.4
1,0,1,Ranger_Archery_SS_Vanilla,3.8
2,0,1,Ranger_TwoCrossbows_NoFeats,5.8
3,0,1,Ranger_TwoCrossbows_SS_Vanilla,6.8
4,0,2,Ranger_Archery_NoFeats,1.0
...,...,...,...,...
239995,4999,11,Ranger_TwoCrossbows_SS_Vanilla,25.8
239996,4999,12,Ranger_Archery_NoFeats,15.4
239997,4999,12,Ranger_Archery_SS_Vanilla,13.8
239998,4999,12,Ranger_TwoCrossbows_NoFeats,17.0


In [10]:
df_overall = df_per_round_stats.groupby(['level', 'name']).agg(
    mean_DPR=('mean_round_damage', 'mean'),
    # median_DPR=('median_round_damage', 'mean'),
    # percentile_10_DPR=('percentile_10_round_damage', 'mean'),
    # percentile_90_DPR=('percentile_90_round_damage', 'mean')
).reset_index()

df_overall

,level,name,mean_DPR
0,1,Ranger_Archery_NoFeats,5.51204
1,1,Ranger_Archery_SS_Vanilla,5.55448
2,1,Ranger_TwoCrossbows_NoFeats,7.19088
3,1,Ranger_TwoCrossbows_SS_Vanilla,7.13468
4,2,Ranger_Archery_NoFeats,5.47604
5,2,Ranger_Archery_SS_Vanilla,5.48256
6,2,Ranger_TwoCrossbows_NoFeats,7.09524
7,2,Ranger_TwoCrossbows_SS_Vanilla,7.07932
8,3,Ranger_Archery_NoFeats,6.56668
9,3,Ranger_Archery_SS_Vanilla,6.58452


In [11]:
import plotly.graph_objects as go
import plotly.express as px


fig = go.Figure()


for name_value in df_overall['name'].unique():
    df_name = df_overall[df_overall['name'] == name_value]
    color = px.colors.qualitative.Set1[df_overall['name'].unique().tolist().index(name_value)]
    
    # fig.add_trace(go.Scatter(
    #     x=df_name['level'],
    #     y=df_name['percentile_90_DPR'],
    #     mode='lines',
    #     name=f'{name_value} DPR 90%',
    #     line=dict(color=color, dash='dash')
    # ))
    # fig.add_trace(go.Scatter(
    #     x=df_name['level'],
    #     y=df_name['percentile_10_DPR'],
    #     mode='lines',
    #     name=f'{name_value} DPR 10%',
    #     line=dict(color=color, dash='dot')
    # ))
    fig.add_trace(go.Scatter(
        x=df_name['level'],
        # y=df_name['median_DPR'],
        y=df_name['mean_DPR'],
        mode='lines+markers',
        name=f'{name_value} Mean DPR',
        line=dict(color=color)
    ))

fig.update_layout(height=800)

fig.show()